In [1]:
import julia
julia.install()

from pathlib import Path

# # Point to the top of the project relative to this script
def projectdir(*args):
    return str(Path.cwd().joinpath("..", "..", "..", *args).resolve())

def print_allocated_memory():
   print("{:.2f} GB".format(torch.cuda.memory_allocated() / 1024 ** 3))

In [2]:
from src.smnistp import SplitMNISTPreprocessed

benchmark = SplitMNISTPreprocessed(
    # n_experiences=5,
    n_experiences=10,
    shuffle=False,
)

In [3]:
from src.ddvfa import DDVFAStrategy

# Create the Strategy Instance
cl_strategy = DDVFAStrategy(projectdir())

# Training Loop
print('Starting experiment...')

results = []
train_results = []
for exp_id, experience in enumerate(benchmark.train_stream):
    print("Start of experience ", experience.current_experience)

    train_results.append(cl_strategy.train(experience))
    print('Training completed')

    print('Computing accuracy on the current test set')
    results.append(cl_strategy.eval(benchmark.test_stream[exp_id]))
    # results.append(cl_strategy.eval(scenario.test_stream))


Starting experiment...
Start of experience  0
5923


100%|██████████| 24/24 [00:06<00:00,  3.71it/s, jl time=0.006] 


Training completed
Computing accuracy on the current test set
980


100%|██████████| 4/4 [00:00<00:00,  7.79it/s, jl time=0.043]


Start of experience  1
6742


...

Training completed
Computing accuracy on the current test set
1032


100%|██████████| 5/5 [00:05<00:00,  1.00s/it, jl time=0.0421]


Start of experience  3
6131


 33%|███▎      | 8/24 [00:17<00:34,  2.15s/it, jl time=2.23]


KeyboardInterrupt: 

In [4]:
print(results, train_results)

[0.0, 1.0, 0.50390625] [0.0, 1.0, 1.0]


In [ ]:
# from torch.utils.data import DataLoader
# from tqdm import tqdm
# import ipdb
# # ipdb.set_trace()
# for exp_id, experience in enumerate(benchmark.train_stream):
#     train_dataset = experience.dataset
#     train_data_loader = DataLoader(
#         dataset=train_dataset,
#         pin_memory=True,
#         # batch_size=90,
#         batch_size=256,
#     )
#     print(experience.dataset.__len__())
#     for mb in tqdm(train_data_loader):
#         data, labels, tasks = mb
#         ipdb.set_trace()
#         # jl.features = self.ext_features(data)
#         # jl.features = data.numpy().transpose()
#         # jl.labels = labels.numpy()
#         # # ipdb.set_trace()
#         # jl.eval("train!(art, features, y=labels)")

12080


  0%|          | 0/48 [00:00<?, ?it/s]

> c:\users\sasha\appdata\local\temp\ipykernel_27684\4086978925.py(14)<module>()

tensor([[[0.0013, 0.0014, 0.0012,  ..., 0.0039, 0.0028, 0.0009]],

        [[0.0007, 0.0006, 0.0003,  ..., 0.0016, 0.0024, 0.0012]],

        [[0.0015, 0.0022, 0.0027,  ..., 0.0011, 0.0007, 0.0004]],

        ...,

        [[0.0017, 0.0016, 0.0006,  ..., 0.0015, 0.0004, 0.0005]],

        [[0.0006, 0.0003, 0.0004,  ..., 0.0002, 0.0002, 0.0004]],

        [[0.0007, 0.0010, 0.0031,  ..., 0.0038, 0.0035, 0.0028]]])
torch.Size([256, 1, 49])
torch.Size([256, 1, 49])


  0%|          | 0/48 [00:17<?, ?it/s]


In [6]:
benchmark.train_stream.__getitem__(0).dataset[0][0].shape

torch.Size([1, 49])

In [5]:
# benchmark.test_stream[1].classes_in_this_experience
# benchmark.train_stream[1].classes_in_this_experience

[8, 5]

In [7]:
from julia import Main as jl
print(jl.eval("AdaptiveResonance.get_n_weights(art)"))
print(jl.eval("size(art.F2)"))
print(jl.eval("art.labels"))

5602
(5602,)
[1 2 3 ... 2 3 2]
